In [1]:
import json
import pandas as pd
import scipy
import time
import seaborn as sns
import random
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import json
import urllib.request
from lxml import html
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from datetime import datetime

!pip install webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.set(rc={'figure.figsize':(11.7,8.27)})
np.random.seed(seed=123)
pd.options.display.float_format = '{:,.2f}'.format

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


Functions

In [7]:
def describe_f(df):
    return display(df.dtypes), display(df.isna().value_counts()), display(df.describe()), \
    display(df), print('----------------------------------------------\n----------------------------------------------\n')

def append_data(j, views, date, link, comments, link_to_blogger, blogger_name, link_data):
    link_date_views = []
    data = {"j": j, 'views': views, 'date': date, 'link': link, 'comments': comments, 
        'link_to_blogger': f"https://www.youtube.com{link_to_blogger}", 'blogger_name': blogger_name}
    link_date_views.append(data)
    pd.DataFrame.from_dict(link_date_views).to_csv(link_data, mode='a', index=False, header=False)   
    print("Данные по блоггеру", link_date_views[-1], '\n============================\n')

**1. j: Видео канала**

**2. views: Просмотры видео**

**3. date: Дата**

**4. link: Код от того, кто пригласил реферала**

**5. comments: Комментарии**

**6. link_to_blogger: Ссылка на канал**

**7. blogger_name: Канал блоггера**

In [11]:
#read CRM data
#link_crm = 
link_data = r"parser.ex.csv" #данные по каналам блоггеров льются в файл                    
#ref_sponsor = pd.read_excel(link_crm, index_col=[0]).iloc[:-1,:]
#describe_f(ref_sponsor)

#create file for data scraping
pd.DataFrame({"j": [], 'views': [], 'date': [], 'link': [], 'comments': [], 
        'link_to_blogger': [], 'blogger_name': []}).to_csv(link_data, index=False)

In [9]:
%%time
link_google = 'https://www.google.com/'
sponsors_link = ['Q5gljk', '9GeEIb', 'lTPr1Q']#блоггер, его рефссылка
error = []
list_120 = [] #120 error, блоеггеров, которых не оказалось
driver = webdriver.Chrome(ChromeDriverManager().install())

for link in sponsors_link:
    print("------------------------------------------------------------")
    driver.get(link_google)
    time.sleep(2)
    while True:
        try:
            driver.find_element(by=By.XPATH,value='//textarea[@class="gLFyf"]').send_keys(f'garantex.io /invite/{link} site:youtube.com')
            break
        except NoSuchElementException:
            print("Delay in google search")
            driver.refresh()
            time.sleep(5)
    #поиск в гугле
    driver.find_element(by=By.XPATH, value='//div[@class="FPdoLc lJ9FBc"]').click()
    time.sleep(0.25)

    soup_main = BeautifulSoup(driver.page_source, 'html')
    soup_main.find_all(attrs={'class': 'MjjYud'})
    soup_main_str = list(soup_main.find_all(name='div', attrs={'id': 'result-stats'}))[0]
    #relates to "for" loop
    try:
        if list(soup_main.find_all(name='p', attrs={'aria-level':"3", 
                'style': "padding-top:.33em"})[0])[2] == ' ничего не найдено.\xa0':
            print(f"There is no blogger with this code: {link}")
            continue
    except IndexError:
            print("Search results:", re.findall(r'\d+', str(soup_main_str))[0])
    #клик на кнопку Видео
    while NoSuchElementException or AttributeError:
        try:
            time.sleep(0.25)
            driver.find_element(by=By.XPATH, value='//a[@data-hveid="CAEQBA"]').click()
            break
        except (NoSuchElementException, AttributeError):
            time.sleep(0.5)

    blogger_links = []#записи блоггеров с приглашением на биржу
    while True:
        soup = BeautifulSoup(driver.page_source, 'html')
        if ('Не найдено ни одного видео по запросу' in str(soup)) & (int(re.findall(r'\d+', str(soup_main_str))[0]) != 0):
            error.append(link)#поиск дал резы, а поиск по видео нет
        list_a = soup.find_all(attrs={'class': 'MjjYud'}) #атрибут по каждому видео
        for i in range(len(list_a)):
            #если есть класс такой же, но не с видео
            if len(list_a[i].find_all(attrs={'class': 'mgAbYb OSrXXb RES9jf IFnjPb'})) != 0:
                continue
            while True:
                try:
                    time.sleep(0.5)
                    aa00 = list_a[i].find_all(name='a', attrs={'href':re.compile("youtube.com/watch?")}, limit=None)
                    #print(aa00[0]['href'])
                    blogger_links.append(aa00[0]['href'])
                    break
                except IndexError:
                    time.sleep(0.5)
                    #print("000")
                    soup = BeautifulSoup(driver.page_source, 'html')
                    list_a = soup.find_all(attrs={'class': 'MjjYud'}) #атрибут по каждому видео
        #есть ли следующая страница
        if len(soup.find_all(name='span', attrs={'style': 'display:block;margin-left:53px'})) != 0:
            driver.find_element(by=By.XPATH, value='//span[@style="display:block;margin-left:53px"]').click()
            continue
        else:
            break
    #записи блоггеров с приглашением на биржу
    print(len(blogger_links), "channel videos with this code", '\n')
    for j in (blogger_links):
        driver.get(j)
        while True:
            try:
                time.sleep(0.25)
                driver.find_element(by=By.XPATH, value='//tp-yt-paper-button[@id="expand"]').click()
                #нажать раскрыть инфо по видео
                time.sleep(0.25)
                break
            except NoSuchElementException:
                #print("задержка по youtube")
                time.sleep(0.5)
        soup_youtube = BeautifulSoup(driver.page_source, 'html')
        
        #search by letters
        str_l = str(soup_youtube.find_all(name='yt-formatted-string', attrs={"id": "info", 
                    'class': 'style-scope ytd-watch-metadata'}, limit=None)[0])
        start_digit = re.search(string=str_l, pattern=re.compile('bold">')).end()
        end_digit = re.search(string=str_l, pattern=re.compile(' views')).end()
        views = str_l[start_digit:end_digit]
        date = (list(soup_youtube.find_all(name='span', attrs={'class': 'style-scope yt-formatted-string bold',
                                                 'style-target': 'bold'}, limit=None)[2])[0])
        link_to_blogger = soup_youtube.find_all(name='a', attrs={'class': "yt-simple-endpoint style-scope yt-formatted-string",
                              'spellcheck': 'false', 'dir':  'auto', 'href':re.compile("/@")}, limit=None)[0]["href"]
        blogger_name = list(soup_youtube.find_all(name='a', attrs={'class': "yt-simple-endpoint style-scope yt-formatted-string",
                              'spellcheck': 'false', 'dir':  'auto', 'href':re.compile("/@")}, limit=None)[0])[0]
        
        height = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script("window.scrollTo(0, " + str(height - 1_000) + ");")
        time.sleep(0.1)
        #Wait to load page
        while True:
            try:
                time.sleep(0.25)
                soup_youtube = BeautifulSoup(driver.page_source, 'html')
                comments_all = list(soup_youtube.find_all(name='span', attrs={'dir': 'auto', 'class': 'style-scope yt-formatted-string'}, limit=None))
                ii = 0
                if f'/invite/{link}' in str(soup_youtube):
                    print("\nsponsor_code is here")
                else:
                    print("120 error\n")
                    list_120.append(j)
                while ('Comment' not in str(comments_all)):
                    #print("len increasing")
                    height = driver.execute_script("return document.documentElement.scrollHeight")
                    driver.execute_script("window.scrollTo(0, " + str(height - 1_200) + ");")
                    time.sleep(0.5)
                    soup_youtube = BeautifulSoup(driver.page_source, 'html')
                    comments_all = list(soup_youtube.find_all(name='span', attrs={'dir': 'auto', 'class': 'style-scope yt-formatted-string'}, limit=None))
                    ii = ii + 1
                    if ii == 7:
                        print("comments are not found")
                        driver.refresh()
                        time.sleep(1.5)
                        #отображение комментариев
                        driver.execute_script("window.scrollTo(0, " + str(height - random.uniform(0, height)) + ");")
                        ii = 0
                if len(comments_all) == 1:
                    comments = re.findall(r'\d+', str(comments_all))[0]
                    print("comments are here (0):", comments)
                elif len(comments_all) > 1:
                    for q in range(len(comments_all)):
                        try:
                            if 'Comment' in str(comments_all[q]):
                                #print('q = ', q)
                                #print(comments[q-1])
                                comments = comments_all[q-1]
                                print(list(comments)[0], " comments\n")
                                comments = list(comments)[0]
                                break
                        except KeyError:
                            print("123 error")
                #print("break error1")
                break
            except IndexError:
                print("Video description error")
                height = driver.execute_script("return document.documentElement.scrollHeight")
                driver.execute_script("window.scrollTo(0, " + str(height - 800) + ");")
                time.sleep(0.5)
        append_data(j, views, date, link, comments, link_to_blogger, blogger_name, link_data)
driver.quit() #выход из браузера

<timed exec>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


------------------------------------------------------------
Search results: 10
11 channel videos with this code 


sponsor_code is here
234  comments

Данные по блоггеру {'j': 'https://www.youtube.com/watch?v=c2LMZRUJ_CE', 'views': '35,546 views', 'date': 'Jan 16, 2023', 'link': 'Q5gljk', 'comments': '234', 'link_to_blogger': 'https://www.youtube.com/@Sosed62', 'blogger_name': 'Сосед'} 


sponsor_code is here
5  comments

Данные по блоггеру {'j': 'https://www.youtube.com/watch?v=l72K3f-49VA', 'views': '4,153 views', 'date': 'Feb 10, 2023', 'link': 'Q5gljk', 'comments': '5', 'link_to_blogger': 'https://www.youtube.com/@Sosed62', 'blogger_name': 'Сосед'} 


sponsor_code is here
14  comments

Данные по блоггеру {'j': 'https://www.youtube.com/watch?v=Jk4VWoHSMu8', 'views': '17,965 views', 'date': 'Feb 4, 2023', 'link': 'Q5gljk', 'comments': '14', 'link_to_blogger': 'https://www.youtube.com/@Sosed62', 'blogger_name': 'Сосед'} 


sponsor_code is here
18  comments

Данные по блоггеру {'j': '

Data preparation

In [10]:
blogger_data_csv = pd.read_csv(link_data, index_col='link')
describe_f(blogger_data_csv)

dict_repl = {"views": '', ',': ''}
blogger_data_csv["comments"] = blogger_data_csv["comments"].astype(str).replace(",", '').astype(int)
blogger_data_csv["views"] = blogger_data_csv["views"].replace(list(dict_repl.keys()), 
                            list(dict_repl.values()), regex=True).astype(int)

#engagement rate
blogger_data_csv['ER'] = 100 * blogger_data_csv["comments"] / blogger_data_csv["views"]
blogger_data_csv = blogger_data_csv.sort_values(by='ER', ascending=False) \
.sort_values(by='ER', ascending=False)

blogger_data_csv

j                  object
views              object
date               object
comments            int64
link_to_blogger    object
blogger_name       object
dtype: object

j      views  date   comments  link_to_blogger  blogger_name
False  False  False  False     False            False           11
dtype: int64

,comments
count,11.00
mean,121.27
std,175.21
min,4.00
25%,10.50
50%,18.00
75%,177.50
max,583.00


,j,views,date,comments,link_to_blogger,blogger_name
link,,,,,,
Q5gljk,https://www.youtube.com/watch?v=c2LMZRUJ_CE,"35,546 views","Jan 16, 2023",234,https://www.youtube.com/@Sosed62,Сосед
Q5gljk,https://www.youtube.com/watch?v=l72K3f-49VA,"4,153 views","Feb 10, 2023",5,https://www.youtube.com/@Sosed62,Сосед
Q5gljk,https://www.youtube.com/watch?v=Jk4VWoHSMu8,"17,965 views","Feb 4, 2023",14,https://www.youtube.com/@Sosed62,Сосед
Q5gljk,https://www.youtube.com/watch?v=V24i_mFoYRo,"15,255 views","Jan 31, 2023",18,https://www.youtube.com/@Sosed62,Сосед
Q5gljk,https://www.youtube.com/watch?v=qXcHNLFz4Ww,"4,190 views","Feb 14, 2023",4,https://www.youtube.com/@Sosed62,Сосед
Q5gljk,https://www.youtube.com/watch?v=rY_S76Jyrt8,"5,390 views","Feb 12, 2023",12,https://www.youtube.com/@Sosed62,Сосед
Q5gljk,https://www.youtube.com/watch?v=J28kIzogr9o,"5,851 views","Feb 6, 2023",9,https://www.youtube.com/@Sosed62,Сосед
Q5gljk,https://www.youtube.com/watch?v=kf3k7ZGOrJY,"16,655 views","May 10, 2023",100,https://www.youtube.com/@Sosed62,Сосед
Q5gljk,https://www.youtube.com/watch?v=p8owdSxKS5U,"31,674 views","Jan 30, 2023",165,https://www.youtube.com/@Sosed62,Сосед


----------------------------------------------
----------------------------------------------



,j,views,date,comments,link_to_blogger,blogger_name,ER
link,,,,,,,
Q5gljk,https://www.youtube.com/watch?v=c2LMZRUJ_CE,35546,"Jan 16, 2023",234,https://www.youtube.com/@Sosed62,Сосед,0.66
Q5gljk,https://www.youtube.com/watch?v=kf3k7ZGOrJY,16655,"May 10, 2023",100,https://www.youtube.com/@Sosed62,Сосед,0.60
Q5gljk,https://www.youtube.com/watch?v=p8owdSxKS5U,31674,"Jan 30, 2023",165,https://www.youtube.com/@Sosed62,Сосед,0.52
Q5gljk,https://www.youtube.com/watch?v=z9WFigRVKxo,52067,"Feb 27, 2023",190,https://www.youtube.com/@Sosed62,Сосед,0.36
Q5gljk,https://www.youtube.com/watch?v=mFPis5sQi74,242101,"Dec 29, 2022",583,https://www.youtube.com/@Sosed62,Сосед,0.24
Q5gljk,https://www.youtube.com/watch?v=rY_S76Jyrt8,5390,"Feb 12, 2023",12,https://www.youtube.com/@Sosed62,Сосед,0.22
Q5gljk,https://www.youtube.com/watch?v=J28kIzogr9o,5851,"Feb 6, 2023",9,https://www.youtube.com/@Sosed62,Сосед,0.15
Q5gljk,https://www.youtube.com/watch?v=l72K3f-49VA,4153,"Feb 10, 2023",5,https://www.youtube.com/@Sosed62,Сосед,0.12
Q5gljk,https://www.youtube.com/watch?v=V24i_mFoYRo,15255,"Jan 31, 2023",18,https://www.youtube.com/@Sosed62,Сосед,0.12
